In [1]:
from dotmap import DotMap
import pandas as pd
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision

from tools.tree_array_transform import VSC, Comp, Range, todf
from tools.trust_ncg import minimize
import tools.che as che


In [5]:
R=8.314 # J/(mol K)
p = che.Props(['Nitrogen','Oxygen', 'Argon', 'Water'])

def model(c):
    Pw = p.Pvap(c.T)[3]
    V_vap = s.V_tot - c.Vw_i
    W_n_vap = Pw * V_vap / (R * c.T)
    n_vap = jnp.concatenate((c.air_n, W_n_vap))
    P = jnp.sum(n_vap) * R * c.T / V_vap
    return (s.P_f - P)**2

s=DotMap()
s.air_y = jnp.array([0.78, 0.21, 0.01])
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.q = 2000. # watts
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.air_y * s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(350, s.T_i, 500.)

vsc=VSC(v,s)
model_f = vsc.transform(model)
model_f(vsc.x)

1668915695.701354

In [6]:
modelf = jax.jit(vsc.transform(model))
x_min, f = minimize(modelf, vsc.x, verbosity=1)

0:0, f: 1668915695.7013612
1:0, f: 1668915695.7013612
2:0, f: 269252825.8254914
3:0, f: 63420571.555923015
4:0, f: 723406.8608923925
5:1, f: 134.00375334309797
6:0, f: 4.813851064645867e-06
Final results:
f: 4.813851064645867e-06
x: [-0.6845626]


In [7]:
v_sol = vsc.xtov(x_min)
v_sol.pprint()





{'T': DeviceArray([365.71923532], dtype=float64)}
